In [ ]:
import gridstatus

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
eia = gridstatus.EIA(api_key=api_key)

In [ ]:
eia.list_routes()

{'id': '',
 'name': '',
 'description': '',
 'routes': [{'id': 'coal',
   'name': 'Coal',
   'description': 'EIA coal energy data'},
  {'id': 'crude-oil-imports',
   'name': 'Crude Oil Imports',
   'description': 'Crude oil imports by country to destination, \r\n        includes type, grade, quantity.  Source: EIA-814  Interactive data \r\n        product:  www.eia.gov/petroleum/imports/companylevel/'},
  {'id': 'electricity',
   'name': 'Electricity',
   'description': 'EIA electricity survey data'},
  {'id': 'international',
   'name': 'International',
   'description': 'Country level production, consumption, imports, exports by energy source (petroleum, natural gas, electricity, renewable, etc.)  \r\n        Interactive product:  https://www.eia.gov/international/data/world'},
  {'id': 'natural-gas',
   'name': 'Natural Gas',
   'description': 'EIA natural gas survey data'},
  {'id': 'nuclear-outages',
   'name': 'Nuclear Outages',
   'description': 'EIA nuclear outages survey data'

In [ ]:
eia.list_routes("electricity")

{'id': 'electricity',
 'name': 'Electricity',
 'description': 'EIA electricity survey data',
 'routes': [{'id': 'retail-sales',
   'name': 'Electricity Sales to Ultimate Customers',
   'description': 'Electricity sales to ultimate customer by state and sector (number of customers, average price, revenue, and megawatthours of sales).  \n    Sources: Forms EIA-826, EIA-861, EIA-861M'},
  {'id': 'electric-power-operational-data',
   'name': 'Electric Power Operations (Annual and Monthly)',
   'description': 'Monthly and annual electric power operations by state, sector, and energy source.\n    Source: Form EIA-923'},
  {'id': 'rto',
   'name': 'Electric Power Operations (Daily and Hourly)',
   'description': 'Hourly and daily electric power operations by balancing authority.  \n    Source: Form EIA-930'},
  {'id': 'state-electricity-profiles',
   'name': 'State Specific Data',
   'description': 'State Specific Data'},
  {'id': 'operating-generator-capacity',
   'name': 'Inventory of Opera

In [ ]:
eia.list_routes("electricity/rto/")

{'id': 'rto',
 'name': 'Electric Power Operations (Daily and Hourly)',
 'description': 'Hourly and daily electric power operations by balancing authority.  \n    Source: Form EIA-930',
 'routes': [{'id': 'region-data',
   'name': 'Hourly Demand, Demand Forecast, Generation, and Interchange',
   'description': 'Hourly demand, day-ahead demand forecast, net generation, and interchange by balancing authority.  \n    Source: Form EIA-930\n    Product: Hourly Electric Grid Monitor'},
  {'id': 'fuel-type-data',
   'name': 'Hourly Generation by Energy Source',
   'description': 'Hourly net generation by balancing authority and energy source.  \n    Source: Form EIA-930\n    Product: Hourly Electric Grid Monitor'},
  {'id': 'region-sub-ba-data',
   'name': 'Hourly Demand by Subregion',
   'description': 'Hourly demand by balancing authority subregion.  \n    Source: Form EIA-930\n    Product: Hourly Electric Grid Monitor'},
  {'id': 'interchange-data',
   'name': 'Hourly Interchange by Neighbo

In [ ]:
eia.list_routes("electricity/rto/interchange-data")

{'id': 'interchange-data',
 'name': 'Hourly Interchange by Neighboring Balancing Authority',
 'description': 'Daily interchange between neighboring balancing authorities.  \n    Source: Form EIA-930\n    Product: Hourly Electric Grid Monitor',
 'frequency': [{'id': 'hourly',
   'alias': 'hourly (UTC)',
   'description': 'One data point for each hour in UTC time.',
   'query': 'H',
   'format': 'YYYY-MM-DD"T"HH24'},
  {'id': 'local-hourly',
   'alias': 'hourly (Local Time Zone)',
   'description': 'One data point for each hour in local time.',
   'query': 'LH',
   'format': 'YYYY-MM-DD"T"HH24TZH'}],
 'facets': [{'id': 'fromba', 'description': 'Balancing Authority / Region'},
  {'id': 'toba', 'description': 'Neighboring Balancing Authority / Region'}],
 'data': {'value': {'aggregation-method': 'SUM',
   'alias': 'Interchange',
   'units': 'megawatthours'}},
 'startPeriod': '2015-07-01T05',
 'endPeriod': '2023-04-27T07',
 'defaultDateFormat': 'YYYY-MM-DD"T"HH24',
 'defaultFrequency': 'hou

In [ ]:
df = eia.get_dataset(
    dataset="electricity/rto/interchange-data",
    start="2023-03-25T00",
    end=None,
    n_workers=4,
    verbose=True,
)

Fetching data from https://api.eia.gov/v2/electricity/rto/interchange-data/data/
Params: {'start': '2020-03-25T00', 'end': None, 'frequency': 'hourly', 'data': ['value'], 'facets': {}, 'offset': 0, 'length': 5000}
Total records: 9265096
Total pages: 1854
Fetching data:


 66%|████████████████████████▎            | 1217/1854 [1:30:34<47:24,  4.47s/it]


KeyboardInterrupt: 

In [ ]:
all_exports = a[a["From BA"] == "CISO"]
all_imports = a[a["To BA"] == "CISO"]

sum_exports = all_exports.groupby("Interval Start").sum()
sum_imports = all_imports.groupby("Interval Start").sum()

import pandas as pd

df = pd.concat([sum_exports, sum_imports], axis=1)
df.columns = ["Exports", "Imports"]
df["Diff"] = df["Exports"] + df["Imports"]
df["% off"] = df["Diff"] / df["Exports"] * 100
df

,Exports,Imports,Diff,% off
Interval Start,,,,
2023-03-24 23:00:00+00:00,-1694,1648,-46,2.715466
2023-03-25 00:00:00+00:00,-2117,1956,-161,7.605102
2023-03-25 01:00:00+00:00,-5063,4836,-227,4.483508
2023-03-25 02:00:00+00:00,-7046,6858,-188,2.668181
2023-03-25 03:00:00+00:00,-7846,7542,-304,3.874586
...,...,...,...,...
2023-04-27 02:00:00+00:00,-4497,4522,25,-0.555926
2023-04-27 03:00:00+00:00,-4224,4225,1,-0.023674
2023-04-27 04:00:00+00:00,-4545,4511,-34,0.748075


In [ ]:
eia.list_routes("electricity/rto/interchange-data")

{'id': 'interchange-data',
 'name': 'Hourly Interchange by Neighboring Balancing Authority',
 'description': 'Daily interchange between neighboring balancing authorities.  \n    Source: Form EIA-930\n    Product: Hourly Electric Grid Monitor',
 'frequency': [{'id': 'hourly',
   'alias': 'hourly (UTC)',
   'description': 'One data point for each hour in UTC time.',
   'query': 'H',
   'format': 'YYYY-MM-DD"T"HH24'},
  {'id': 'local-hourly',
   'alias': 'hourly (Local Time Zone)',
   'description': 'One data point for each hour in local time.',
   'query': 'LH',
   'format': 'YYYY-MM-DD"T"HH24TZH'}],
 'facets': [{'id': 'fromba', 'description': 'Balancing Authority / Region'},
  {'id': 'toba', 'description': 'Neighboring Balancing Authority / Region'}],
 'data': {'value': {'aggregation-method': 'SUM',
   'alias': 'Interchange',
   'units': 'megawatthours'}},
 'startPeriod': '2015-07-01T05',
 'endPeriod': '2023-04-27T07',
 'defaultDateFormat': 'YYYY-MM-DD"T"HH24',
 'defaultFrequency': 'hou

In [ ]:
a

,Interval Start,Interval End,From BA,From BA Name,To BA,To BA Name,MW
45125,2023-03-24 23:00:00+00:00,2023-03-25 00:00:00+00:00,AECI,"Associated Electric Cooperative, Inc.",SPA,Southwestern Power Administration,-237
47344,2023-03-24 23:00:00+00:00,2023-03-25 00:00:00+00:00,AECI,"Associated Electric Cooperative, Inc.",MISO,"Midcontinent Independent System Operator, Inc.",156
60800,2023-03-24 23:00:00+00:00,2023-03-25 00:00:00+00:00,AECI,"Associated Electric Cooperative, Inc.",SWPP,Southwest Power Pool,106
60831,2023-03-24 23:00:00+00:00,2023-03-25 00:00:00+00:00,AECI,"Associated Electric Cooperative, Inc.",TVA,Tennessee Valley Authority,90
45156,2023-03-24 23:00:00+00:00,2023-03-25 00:00:00+00:00,AVA,Avista Corporation,CHPD,Public Utility District No. 1 of Chelan County,12
...,...,...,...,...,...,...,...
219458,2023-04-27 06:00:00+00:00,2023-04-27 07:00:00+00:00,WALC,Western Area Power Administration - Desert Sou...,IID,Imperial Irrigation District,-7
227881,2023-04-27 06:00:00+00:00,2023-04-27 07:00:00+00:00,WALC,Western Area Power Administration - Desert Sou...,LDWP,Los Angeles Department of Water and Power,932
231121,2023-04-27 06:00:00+00:00,2023-04-27 07:00:00+00:00,WALC,Western Area Power Administration - Desert Sou...,NEVP,Nevada Power Company,156
231144,2023-04-27 06:00:00+00:00,2023-04-27 07:00:00+00:00,WALC,Western Area Power Administration - Desert Sou...,AZPS,Arizona Public Service Company,-928
